# Community detection

**Community detection** is the process of dividing a network into **communities**, where each community is some subset of nodes which connect more within community than without.
This is an old and famous network science problem. It has applications across a wide
range of disciplines. To name a few:
- Predicting "workgroups" in organizational networks {cite}`zuzul2021dynamic`